In [29]:
import pandas as pd 
import json
from collections import defaultdict

In [74]:
disease_df = pd.read_csv("diseases_data.csv")
desc_df = pd.read_csv("diseases_desc.csv")


symptoms = set()


# # dump into file 
# with open("symptoms.json","w") as file:
#     file.write(json.dumps(list(symptoms)))

data_dict = disease_df.to_dict()
key_list = list(data_dict.keys())[1:]

for key in key_list:
    key_values = data_dict[key].values()
    for symptom in key_values:
        if isinstance(symptom,str):
            symptom = symptom.strip().replace("_"," ").capitalize()
            symptoms.add(symptom)

disease_map = defaultdict(list)

    
d_names = [name.strip().capitalize() for name in data_dict["Disease"].values()]

for col in disease_df.columns:
    disease_df[col] = [cell.strip().replace("_"," ").capitalize() if isinstance(cell,str) else "N/A" for cell in disease_df[col]]

for col in desc_df.columns:
    desc_df[col] = [cell.strip().replace("_"," ").capitalize() if isinstance(cell,str) else "N/A" for cell in desc_df[col]]

symptom_map = defaultdict(list)

for row in disease_df.itertuples():
    filtered_attr = filter(lambda attr : "Symptom_" in attr,dir(row))
    for attr in filtered_attr:
        symptom_map[row.Disease].append(getattr(row,attr))


final_map = defaultdict(dict)

symptom_map = {key : list(set(filter(lambda val : val != "N/A",value))) for key,value in symptom_map.items()}

for row in desc_df.itertuples():
    final_map[row.Disease]["desc"] = row.Description

for key in final_map:
    if key in symptom_map:
        final_map[key]["symptoms"] = symptom_map[key]



new_df = pd.DataFrame() 

new_df["Disease"] = [key for key in final_map]

with open("diseases.json","r") as file:
    final_map = json.loads(file.read())

for sym in symptoms:
    new_df[sym] = [1 if sym in final_map[key]["symptoms"] else 0 for key in final_map]

new_df.to_csv("final_disease_data.csv",)

C:\Users\Conne\AppData\Local\Temp/ipykernel_17620/2642781186.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[sym] = [1 if sym in final_map[key]["symptoms"] else 0 for key in final_map]
